In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

In [2]:
directory = "../input/m5-forecasting-accuracy/"
sample_submission = pd.read_csv(directory + "sample_submission.csv")
sales_train_validation = pd.read_csv(directory + "sales_train_validation.csv")
calendar = pd.read_csv(directory + "calendar.csv")
sell_prices = pd.read_csv(directory + "sell_prices.csv")

In [3]:
calendar.tail(10)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1959,2016-06-10,11619,Friday,7,6,2016,d_1960,NaN,NaN,NaN,NaN,1,0,0
1960,2016-06-11,11620,Saturday,1,6,2016,d_1961,NaN,NaN,NaN,NaN,0,1,1
1961,2016-06-12,11620,Sunday,2,6,2016,d_1962,NaN,NaN,NaN,NaN,0,1,1
1962,2016-06-13,11620,Monday,3,6,2016,d_1963,NaN,NaN,NaN,NaN,0,1,0
1963,2016-06-14,11620,Tuesday,4,6,2016,d_1964,NaN,NaN,NaN,NaN,0,0,1
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [4]:
weekends = calendar[calendar['wday'] <= 2]['d']
weekdays = calendar[calendar['wday'] >= 3]['d']

cuttoff_day = 1880
weekends_cutoff = [day for day in weekends if int(day[2:]) > cuttoff_day]
weekdays_cutoff = [day for day in weekdays if int(day[2:]) > cuttoff_day]

weekends_train = [day for day in weekends_cutoff if day in sales_train_validation.columns]
weekdays_train = [day for day in weekdays_cutoff if day in sales_train_validation.columns]

In [5]:
weekend_sales = sales_train_validation.set_index('id')[weekends_train]
weekend_median_sales = weekend_sales.median(axis=1)
weekend_average_sales = weekend_sales.mean(axis=1)
print("Weekend Median Sales")
print(weekend_median_sales)
print("Weekday Median Sales")
print(weekend_average_sales)


Weekend Median Sales
id
HOBBIES_1_001_CA_1_validation    1.0
HOBBIES_1_002_CA_1_validation    0.0
HOBBIES_1_003_CA_1_validation    1.0
HOBBIES_1_004_CA_1_validation    4.5
HOBBIES_1_005_CA_1_validation    1.0
                                ... 
FOODS_3_823_WI_3_validation      0.0
FOODS_3_824_WI_3_validation      0.5
FOODS_3_825_WI_3_validation      1.0
FOODS_3_826_WI_3_validation      1.0
FOODS_3_827_WI_3_validation      0.0
Length: 30490, dtype: float64
Weekday Median Sales
id
HOBBIES_1_001_CA_1_validation    1.1
HOBBIES_1_002_CA_1_validation    0.2
HOBBIES_1_003_CA_1_validation    0.7
HOBBIES_1_004_CA_1_validation    3.8
HOBBIES_1_005_CA_1_validation    1.5
                                ... 
FOODS_3_823_WI_3_validation      0.1
FOODS_3_824_WI_3_validation      0.5
FOODS_3_825_WI_3_validation      0.9
FOODS_3_826_WI_3_validation      1.5
FOODS_3_827_WI_3_validation      1.0
Length: 30490, dtype: float64


In [6]:
weekday_sales = sales_train_validation.set_index('id')[weekdays_train]
weekday_median_sales = weekday_sales.median(axis=1)
weekday_average_sales = weekday_sales.mean(axis=1)
print("Weekday Median Sales:")
print(weekday_median_sales)
print("Weekday Average Sales:")
print(weekday_average_sales)

Weekday Median Sales:
id
HOBBIES_1_001_CA_1_validation    1.0
HOBBIES_1_002_CA_1_validation    0.0
HOBBIES_1_003_CA_1_validation    0.0
HOBBIES_1_004_CA_1_validation    1.0
HOBBIES_1_005_CA_1_validation    1.0
                                ... 
FOODS_3_823_WI_3_validation      0.0
FOODS_3_824_WI_3_validation      0.0
FOODS_3_825_WI_3_validation      0.0
FOODS_3_826_WI_3_validation      1.0
FOODS_3_827_WI_3_validation      0.0
Length: 30490, dtype: float64
Weekday Average Sales:
id
HOBBIES_1_001_CA_1_validation    0.913043
HOBBIES_1_002_CA_1_validation    0.173913
HOBBIES_1_003_CA_1_validation    0.478261
HOBBIES_1_004_CA_1_validation    1.434783
HOBBIES_1_005_CA_1_validation    1.043478
                                   ...   
FOODS_3_823_WI_3_validation      0.217391
FOODS_3_824_WI_3_validation      0.173913
FOODS_3_825_WI_3_validation      0.826087
FOODS_3_826_WI_3_validation      0.913043
FOODS_3_827_WI_3_validation      1.521739
Length: 30490, dtype: float64


In [7]:
# F1 is d_1914
# F28 is d_1941

weekend_numbers = [int(weekend[2:]) for weekend in weekends]
forecast_weekends = ['F{}'.format(wn - 1913) for wn in weekend_numbers if (wn >= 1914 and wn <= 1941)]
weekday_numbers = [int(weekday[2:]) for weekday in weekdays]
forecast_weekdays = ['F{}'.format(wn - 1913) for wn in weekday_numbers if (wn >= 1914 and wn <= 1941)]

In [8]:
median_submission = sample_submission.copy()
mean_submission = sample_submission.copy()
print("Done copying")
for i in range(sales_train_validation.shape[0]):
    if i % 1000 == 0:
        print("Row: {}".format(i))
    id_ = sample_submission.loc[i, 'id']
    median_submission.loc[i, forecast_weekends] = weekend_median_sales[id_]
    median_submission.loc[i, forecast_weekdays] = weekday_median_sales[id_]
    mean_submission.loc[i, forecast_weekends] = weekend_average_sales[id_]
    mean_submission.loc[i, forecast_weekdays] = weekday_average_sales[id_]

Done copying
Row: 0
Row: 1000
Row: 2000
Row: 3000
Row: 4000
Row: 5000
Row: 6000
Row: 7000
Row: 8000
Row: 9000
Row: 10000
Row: 11000
Row: 12000
Row: 13000
Row: 14000
Row: 15000
Row: 16000
Row: 17000
Row: 18000
Row: 19000
Row: 20000
Row: 21000
Row: 22000
Row: 23000
Row: 24000
Row: 25000
Row: 26000
Row: 27000
Row: 28000
Row: 29000
Row: 30000


In [9]:
median_submission.to_csv("median.csv", index=False)
mean_submission.to_csv("average.csv", index=False)